# A simple demondstration of Bayesian Networks

In this notebook, I'm going to use the [Julia](https://julialang.org/) programming language to demonstrate some of the basic ideas of a Bayesian network. We will use the Julia package [BayesNets.jl](https://sisl.github.io/BayesNets.jl/dev/) to build and reason with very simple Bayesian networks.

## Main Assumption

- All our variables are discrete (that is, they can only have a finite number of possible values).

In [ ]:
using BayesNets

In [ ]:
using TikzGraphs # required to plot tex-formatted graphs (recommended), otherwise GraphPlot.jl is used

## Example: Satellite Monitoring

To start with, here is a non-medical example from [_Algorithms for Decision Making_](https://algorithmsbook.com). The objective for the model is to understand the behavior of a satellite which may experience a trajectory deviation due to various components of teh satellite failing. These are the components of the satellite:

- `Battery` battery failure
- `Solar` solar panel failure
- `Electrical` electrical system failure
- `Deviation` trajectory deviation
- `Communication` communication loss.

Here is how the authors describe the model.

>Fortunately, battery failure and solar panel failures are both rare, although solar panel failures are somewhat more likely than battery failures. Failures in either can lead to an electrical system failure. There may be causes of electrical system failure other than battery or solar panel failure, such as a problem with the power management unit. An electrical system failure can result in trajectory deviation, which can be observed from the earth by telescope, as well as a communication loss that interrupts the transmission of telemetry and mission data down to various ground stations. Other anomalies not involving the electrical system can result in trajectory deviation and communication loss. (p. 34)

The Bayesian network for this model looks like this:

<img src="sat.png" alt="satellite model" width="175">


### Specifying the model

For each of the nodes, we need to provide the appropriate probabilities. For nodes without parents, we need to provide the prior probabilities for each state. The states are ordered so the first value represents the probability of not failing and the second value represents the probability of failing. So, for example, with the battery

\begin{eqnarray}
P(\text{not battery fails})=&0.99\\
P(\text{battery fails})=&0.99&=&0.01
\end{eqnarray}

Which we specify with an array passed to the descrete conditional probability distribution function (`DiscreteCPD`).

```Julia
DiscreteCPD(:Battery, [0.99, 0.01])
```

For the nodes with parents, we need to provide all the conditional probabilities. So, for example for Communication loss we need to provide the probabilities of 

1. no communication loss given no electrical failure (0.98)
2. communication loss given no electrical failure (0.02)
3. no communication loss given electrical failure (0.01)
4. communication loss given electrical failure (0.99)

This looks like the following

```Julia

DiscreteCPD(:Communication, [:Electrical], [2],
        [Categorical([0.98, 0.02]),
         Categorical([0.01, 0.99])])
```

In [ ]:
bns = DiscreteBayesNet()
push!(bns, DiscreteCPD(:Battery, [0.99, 0.01]))
push!(bns, DiscreteCPD(:Solar, [0.98, 0.02]))
push!(bns, DiscreteCPD(:Electrical, [:Battery, :Solar], [2,2],
        [Categorical([0.9, 0.1]), # (B=1 & S=1 & E=1, B=1 & S=1 & S=2)
         Categorical([0.05,0.95]), #(B=2 & S=1 & E=1, B=2 & S=1 & E=2)
         Categorical([0.04,0.96]), #(B=1 & S=2 & E=1, B=1 & S=2 & E=2)
         Categorical([0.01,0.99])])) #(B=2 & S=2 & E=1, B=2 & S=2 & E=2)
push!(bns, DiscreteCPD(:Deviation, [:Electrical], [2],
        [Categorical([0.96, 0.04]), #(E=1 & D=1, E=1 & D=2)
         Categorical([0.03, 0.97])])) #(E=2 & D=1, E=2 & D=2)
push!(bns, DiscreteCPD(:Communication, [:Electrical], [2],
        [Categorical([0.98, 0.02]), #(E=1 & C=1, E=1 & C=2)
         Categorical([0.01, 0.99])])); #(E=2 & D=1, E=2 & D=2)


### We can draw the network

In [ ]:
bns

### Checking

We can use the `table` function to check that we have specified the probabilities correctly.

In [ ]:
table(bns, :Electrical)

## Inferencing 

Now that we have our model, we can do inferencing

We start off by calculating what the probabilities of there being no electrical failure (1) or an electrical failure (2) without any evidence.

We can do this for any of our variables we are interested.

In [ ]:
infer(bns, :Electrical)

In [ ]:
infer(bns, :Deviation)

### What is the probability of there being a battery failure given that we have a trajectory deviation?

- We specify what we know using `evidence` and `Assignment`
  - In this case we are assigning the value of 2 (True) to `:Deviation`
- We then `infer` the value we are interested, in this case `:Battery`

In [ ]:
infer(bns, :Battery, evidence=Assignment(:Deviation=>2))

### What is the state of the battery if we have trajectory deviation and we know the solar panel has failed?

In [ ]:
infer(bns, :Battery, evidence=Assignment(:Deviation=>2, :Solar=>2))

### Change the values around and see how the behavior changes
- Remember the values in the square brackets have to add up to 1
- If they don't, you'll get an error like this:


## A medical example

The following is an early example of a Bayesian network provided by S. L. Lauritzen and D. J. Spiegelhalter in their 1988 article ["Local Computations with Probabilities on Graphical Structures and Their Application to Expert Systems"](http://www.jstor.org/stable/2345762?origin=JSTOR-pdf).

Here is the problem as stated by the authors:

>Shortness-of-breath (dyspnoea) may be due to tuberculosis, lung cancer or bronchitis, or none of them, or more than one of them. A recent visit to Asia increases the chances af tuberculosis, while smoking is known to be a risk factor for both lung cancer and bronchitis. The results of a single chest X-ray do not discriminate between lung cancer and tuberculosis, as neither does the presence or absence of dyspnoea. (p. 163)

The authors define the following network topology

![](./media/bn_topology.png)

and provide these probabilities. (Remember, since probabilities have to add up to 1, for binary variables we only need to provide one value.)

![](./media/bn_probabilities.png)

### We can create the network as follows:

In [ ]:
bncxr = DiscreteBayesNet()
push!(bncxr, DiscreteCPD(:Smoker, [0.5, 0.5]))
push!(bncxr, DiscreteCPD(:Asia, [0.99,0.01]))

push!(bncxr, CategoricalCPD(:LungCancer, [:Smoker], [2], 
        [Categorical([0.99,0.01]),
         Categorical([0.90,0.10])]))
push!(bncxr, CategoricalCPD(:Bronchitis, [:Smoker], [2], 
        [Categorical([0.70,0.30]),
         Categorical([0.40,0.60])]))
push!(bncxr, CategoricalCPD(:Tuberculosis, [:Asia], [2], 
        [Categorical([0.99,0.01]),
         Categorical([0.95,0.05])]))
push!(bncxr, DiscreteCPD(:TuberculosisOrCancer, [:Tuberculosis, :LungCancer], [2,2], 
        [Categorical([1.00,0.00]), 
         Categorical([0.00,1.00]), 
         Categorical([0.00,1.00]), 
         Categorical([0.00,1.00]), 
        ]))
push!(bncxr, CategoricalCPD(:CXR, [:TuberculosisOrCancer], [2], 
        [Categorical([0.95,0.05]),
         Categorical([0.02,0.98])]))
push!(bncxr, CategoricalCPD(:Dyspnoea, [:TuberculosisOrCancer, :Bronchitis], [2,2], 
        [Categorical([0.90,0.10]), 
         Categorical([0.30,0.70]), 
         Categorical([0.20,0.80]), 
         Categorical([0.10,0.90]), 
        ]))

In [ ]:
table(bncxr, :CXR)

In [ ]:
infer(bncxr, :LungCancer)

In [ ]:
table(bncxr, :Dyspnoea)

### We can explore the probability of different diseases given different findings

Remember, 1 means false and 2 means true.

In [ ]:
infer(bncxr, :LungCancer, 
    evidence=Assignment(
        :Asia=>1, 
        :Dyspnoea=>2,
        :Smoker=>2,
        :CXR=>2))

In [ ]:
infer(bncxr, :Bronchitis, 
    evidence=Assignment(
        :Asia=>1, 
        :Dyspnoea=>2,
        :Smoker=>1,
        :CXR=>1))

In [ ]:
infer(bncxr, :Tuberculosis, 
    evidence=Assignment(
        :Asia=>2, 
        :Dyspnoea=>2,
        :Smoker=>1,
        :CXR=>1))